In [1]:
import ipyparallel
import hddm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from patsy import dmatrix
%matplotlib inline

print(hddm.__version__)

0.8.0


/home/jasongong/.local/lib/python3.7/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


# load data

### load RTs data

In [6]:
data = hddm.load_csv('data/study1.csv')

data = hddm.utils.flip_errors(data)
data.head(10)

,Unnamed: 0,subj_idx,stim,rt,response,participant
0,1,0,0V0A,3.6243,1,79516
1,2,0,0V0A,-2.5559,0,79516
2,3,0,0V0A,-3.5233,0,79516
3,4,0,0V0A,-1.8405,0,79516
4,5,0,0V0A,-1.8054,0,79516
5,6,0,0V0A,-2.0442,0,79516
6,7,0,0V0A,0.7801,1,79516
7,8,0,0V0A,-2.9547,0,79516
8,9,0,0V0A,3.0110,1,79516
9,10,0,0V0A,1.7631,1,79516


In [7]:
# drop subject 10 due to data convergence issue
data = data.loc[~(data.subj_idx == 10),]

# Confirmatory Model fitting - HDDM model

In [ ]:
# fit the HDDM model with drift rate parameter
# sample 10000 with 2000 burn-in samples
# save the model
m = hddm.HDDM(data, depends_on={'v': 'stim'}, p_outlier=.05)
m.find_starting_values()
m.sample(10000, burn=2000, dbname='traces_hddm.db', db='pickle')
m.save('traces_hddm')

# Test correlation with measured mood

In [13]:
# load data for measured mood for each participants
mood = pd.read_csv("data/study1_measured_mood.csv")

In [16]:
# data wrangling
subj_index_0V1A = [index for index in list(m.nodes_db.index) if "v_subj(0V1A)" in index]
subj_index_1V0A = [index for index in list(m.nodes_db.index) if "v_subj(1V0A)" in index]
node_0V1A = m.nodes_db.loc[subj_index_0V1A, 'node']
node_1V0A = m.nodes_db.loc[subj_index_1V0A, 'node']
v_0V1A_subj = []
v_1V0A_subj = []
for i in range(len(node_0V1A)):
    v_0V1A_subj.append(node_0V1A[i].trace().mean())
    v_1V0A_subj.append(node_1V0A[i].trace().mean())
mood["v_0V1A"] = v_0V1A_subj
mood["v_1V0A"] = v_1V0A_subj

In [17]:
mood

,Unnamed: 0,subj_idx,movie_valence_measured,movie_arousal_measured,v_0V1A,v_1V0A
0,0,0,3,1,0.331669,-0.543508
1,140,1,3,2,-0.087603,-0.043280
2,280,2,4,2,-0.137881,-0.322720
3,420,3,3,2,-0.007891,0.250543
4,560,4,3,3,-0.180247,-0.150996
...,...,...,...,...,...,...
135,19040,155,5,3,-0.156974,0.198980
136,19180,156,4,2,0.047843,-0.182116
137,19320,158,3,1,0.012487,-0.547097
138,19460,160,3,2,0.005649,-0.337632


In [18]:
import pymc3 as pm

In [19]:
# test the correlation between drift rate and measured mood 
# using pymc3 package
# test valence

with pm.Model() as model_valence:
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta_valence = pm.Normal("beta", mu=0, sigma=10)
    sigma = pm.HalfNormal("sigma", sigma=1)
    
    mu_valence = alpha + beta_valence * mood["movie_valence_measured"].values
    
    v_1V0A = pm.Normal('v_1V0A', mu=mu_valence, sd=sigma, observed=mood['v_1V0A'].values)
    
    trace_v = pm.sample(10000, tune=2000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, beta, alpha]


Sampling 4 chains for 2_000 tune and 10_000 draw iterations (8_000 + 40_000 draws total) took 16 seconds.


In [20]:
# test the correlation between drift rate and measured mood 
# using pymc3 package
# test arousal
with pm.Model() as model_arousal:
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta_arousal = pm.Normal("beta", mu=0, sigma=10)
    sigma = pm.HalfNormal("sigma", sigma=1)
    
    mu_arousal = alpha + beta_arousal * mood["movie_arousal_measured"].values
    
    v_0V1A = pm.Normal('v_0V1A', mu=mu_arousal, sd=sigma, observed=mood['v_0V1A'].values)
    
    trace_a = pm.sample(10000, tune=2000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, beta, alpha]


Sampling 4 chains for 2_000 tune and 10_000 draw iterations (8_000 + 40_000 draws total) took 14 seconds.
The acceptance probability does not match the target. It is 0.8788659248551624, but should be close to 0.8. Try to increase the number of tuning steps.


In [24]:
# save the traces of the correlation to csv for inference testing and plotting
pd.DataFrame({"cor_valence": trace_v['beta'],
              "cor_arousal": trace_a['beta']}).to_csv("plot/study1_correlation_traces.csv")

# Exploratory Full Regression Analysis

In [11]:
m_full_model = hddm.HDDMRegressor(data, ["v ~ movie_valence + movie_arousal + movie_valence*movie_arousal",
                                   "a ~ movie_valence + movie_arousal + movie_valence*movie_arousal",
                                   "t ~ movie_valence + movie_arousal + movie_valence*movie_arousal",
                                   "z ~ movie_valence + movie_arousal + movie_valence*movie_arousal"],
                            include=('v', 'a', 't', 'z'),
                            p_outlier=0.05)

Adding these covariates:
['v_Intercept', 'v_movie_valence[T.1]', 'v_movie_arousal[T.1]', 'v_movie_valence[T.1]:movie_arousal[T.1]']
Adding these covariates:
['a_Intercept', 'a_movie_valence[T.1]', 'a_movie_arousal[T.1]', 'a_movie_valence[T.1]:movie_arousal[T.1]']
Adding these covariates:
['t_Intercept', 't_movie_valence[T.1]', 't_movie_arousal[T.1]', 't_movie_valence[T.1]:movie_arousal[T.1]']
Adding these covariates:
['z_Intercept', 'z_movie_valence[T.1]', 'z_movie_arousal[T.1]', 'z_movie_valence[T.1]:movie_arousal[T.1]']


In [ ]:
m_full_model.find_starting_values()
m_full_model.sample(5000, burn=1000, dbname='traces_regression.db', db='pickle')
m_full_model.save('traces_regression')